In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import concurrent.futures
import pandas as pd

In [77]:
url = 'https://www.ourlads.com/ncaa-football-depth-charts'

ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")
team_name = soup.find_all('div', class_= "nfl-dc-mm-team-name")
team_link = soup.find_all('div', class_= "nfl-dc-mm-team-links")





In [78]:
team_links = []
team_link = soup.find_all('div', class_= "nfl-dc-mm-team-links")
for i in team_link:
    team_links.append('https://www.ourlads.com/ncaa-football-depth-charts/'+ i.a['href'])

team_links = team_links[1:]
team_links

['https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=army&id=90038',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=charlotte&id=92936',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=east-carolina&id=90452',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=florida-atlantic&id=90521',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=memphis&id=91050',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=navy&id=91257',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=north-texas&id=92660',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=rice&id=91740',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=south-florida&id=91855',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=temple&id=91970',
 'https://www.ourlads.com/ncaa-football-depth-charts/depth-chart.aspx?s=t

In [79]:
def get_all_csv(url):   
    ua = UserAgent()
    userAgent = ua.random
    head = {'User-Agent': userAgent}
    page = requests.get(url, headers = head)
    soup = BeautifulSoup(page.content, "html.parser")
    team_name = soup.find('div', {'class': 'pt-team'}).text.strip()

    table = soup.find('table', class_='table table-bordered')


    # Extract the headers
    headers = [th.text.strip() for th in table.find_all('th')]

    # Extract the rows
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        # Each cell within the row
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    team_row = [team_name] + [''] * (len(headers) - 1)  
    rows.insert(0, team_row)



    df = pd.DataFrame(rows, columns=headers)

    df = df[~df['Pos'].isin(['Offense', 'Defense', 'Special Teams','OFF'])]

    columns = df.columns.tolist()

    # Initialize a counter for the number of inserted columns
    inserted = 0

    # Iterate through the columns
    for i, col in enumerate(columns):
        if col.startswith('Player'):
            # Insert a new 'twitter' column after the 'Player' column
            df.insert(i + 1 + inserted, f'twitter_{inserted + 1}', '')
            inserted += 1

    df.to_csv(f'/workspace/sales/sports/{team_name}.csv',index=False)
    print(f'{team_name} gotten')


for i in team_links:
    get_all_csv(i)

Army Black Knights gotten
Charlotte 49ers gotten
East Carolina Pirates gotten
Florida Atlantic Owls gotten
Memphis Tigers gotten
Navy Midshipmen gotten
North Texas Mean Green gotten
Rice Owls gotten
South Florida Bulls gotten
Temple Owls gotten
Tulane Green Wave gotten
Tulsa Golden Hurricane gotten
UAB Blazers gotten
UTSA Roadrunners gotten
Boston College Eagles gotten
California Golden Bears gotten
Clemson Tigers gotten
Duke Blue Devils gotten
Florida State Seminoles gotten
Georgia Tech Yellow Jackets gotten
Louisville Cardinals gotten
Miami Hurricanes gotten
North Carolina Tar Heels gotten
North Carolina State Wolfpack gotten
Pittsburgh Panthers gotten
SMU Mustangs gotten
Stanford Cardinal gotten
Syracuse Orange gotten
Virginia Cavaliers gotten
Virginia Tech Hokies gotten
Wake Forest Demon Deacons gotten
Illinois Fighting Illini gotten
Indiana Hoosiers gotten
Iowa Hawkeyes gotten
Maryland Terrapins gotten
Michigan Wolverines gotten
Michigan State Spartans gotten
Minnesota Golden Goph

In [80]:
import pandas as pd
import os

# Specify the folder containing the CSV files
folder_path = '/workspace/sales/sports'  # Replace with your folder path

# List all CSV files in the directory
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty DataFrame
merged_df = pd.DataFrame()

# Loop through each CSV file and append it to the merged DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    
    # Append the data to the merged DataFrame
    merged_df = pd.concat([merged_df, df])
    
    # Add three empty rows (filled with empty strings)
    empty_rows = pd.DataFrame([[''] * len(merged_df.columns)] * 3, columns=merged_df.columns)
    merged_df = pd.concat([merged_df, empty_rows])

# Reset the index (optional, to have a clean index)
merged_df.reset_index(drop=True, inplace=True)

# Save the merged DataFrame to a new CSV file
output_file_path = os.path.join(folder_path, 'merged_output.csv')
merged_df.to_csv(output_file_path, index=False)

print("CSV files merged successfully into 'merged_output.csv'!")


CSV files merged successfully into 'merged_output.csv'!
